# Werken met databases en pandas

## Introductie

Een database is een digitaal opgeslagen gegevensbank ingericht met het oog op flexibele raadpleging en gebruik. Om met een database te kunnen werken is een Data Base Management System (DBMS) nodig. Een DBMS is de software die nodig is om databases te maken, te beheren en te gebruiken. Er zijn verschillende soorten databases. Wij werken in de komende labs met een relationele database.

In een relationele database worden gegevens opgeslagen in tabellen. Een tabel bestaat uit kolommen en rijen. Rijen in een tabel worden uniek geïndentificeerd door een primary key. Tabellen kunnen aan elkaar gekoppeld door foreign key kolommen te gebruiken die verwijzen naar de primary key kolom van een andere tabel.

Om met relationale databases te kunnen werken heb je een Relational Data Base Management Systeem (RDBMS) nodig. Wij werken de komende labs met het [open source RDBMS PostgreSQL](https://www.postgresql.org/). Andere veel gebruikte systemen zijn Oracle en SQLite.

Je gebruikt de taal [SQL](https://nl.wikipedia.org/wiki/SQL) (Structured Query Language) om instructies te geven aan RDBMS om bepaalde taken uit te voeren. Er zijn twee soorten SQL commando's:

* DDL (Data Definition Language) om databases en tabellen te maken en beheren
* DML (Data Manipulation Language) om gegevens aan tabellen toe te voegen (insert), gegevens in tabellen bij te werken (update) en gegevens uit tabellen te verwijderen (delete).

Om gegevens uit database tabellen te selecteren gebruik je [SQL queries](https://nl.wikipedia.org/wiki/Select_(SQL)). Met SQL queries selecteer je nul, één of meerdere rijen uit één of meerdere tabellen op basis van nul, één of meerdere condities. 

## Maken van een PostgreSQL database

PostgreSQL is een RDBMS dat op de server draait. Het RDBMS handelt de SQL verzoeken af die binnenkomen vanaf applicaties zoals pgAdmin (voor database beheer), Geoserver (voor publiceren van geodata), QGIS (voor beheer van geodata) of Python scripts (voor maken van tabellen en bewerken van data). PostgreSQL maakt het mogelijk om met grote aantallen gebruikers gelijktijdig (concurrent) gegevens te benaderen en te bewerken. Ook is PostgreSQL goed schaalbaar. Dit betekent dat er geen limiet is aan de grootte van de gegevensverzamelingen waarmee gewerkt kan worden.

Het maken van een PostgreSQL database doen we met een Python script. Het is ook mogelijk om een database te maken van programma's als [pgAdmin](https://www.pgadmin.org/). Om vanuit een Python script te kunnen communiceren met PostgreSQL heb je de module *psycopg2* nodig. Het script begint daarom met het laden van de module *psycopg2*.

In [ ]:
# Basic imports
import os

# Import module for PostgreSQL
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

Als de module geladen is, kan je een verbinding maken met de database server. Hiervoor moet je de volgende parameters weten:
- *host*: Server waar het PostgreSQL RDBMS draait.
- *port*: Port op de server waar het PostgreSQL RDBMS luistert voor binnenkomende verzoeken.
- *user*: Gebruikernaam om mee aan te loggen (_postgres_ is de standaard gebruikersnaam van de superuser).
- *password*: Wachtwoorde van de gebruiker waarmee je aanlogt.

**Let op**: Wij maken gebruik van de superuser _postgres_ zolang we werken met PostgreSQL op onze eigen laptop. Als je met PostgreSQL werkt op een productieomgeving maak je __nooit__ gebruik van de user _postgres_.

In [ ]:
# Database connection parameters
host = 'localhost'
port = '5432'
user = 'postgres'
password = 'postgres'

# Build connect string
db_connect = "user=" + user + " host=" + host + " port=" + port + " password=" + password + '"'

# Make connection with database
conn = psycopg2.connect(db_connect)

# Print success
print('Database connection succeeded')

## Maken van tabellen in een PostgreSQL database

We gaan werken met een relationeel datamodel dat bestaat uit drie tabellen:

* meteostation
* waarneming
* provincie

Het ERD is met de primary keys (PK) en foreign keys (FK) is hieronder hier te bekijken.

Om deze tabellen te kunnen maken, moeten we eerst een database creëren. We gaan een database __meteodb__ creëren voor de opslag van onze data.

In [ ]:
# Set autocommit on
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Set name of database to create
database_name = 'meteodb'

# Build sql statement to create database
sql_stmt = 'CREATE DATABASE ' + database_name

# Get a cursor to execute SQL statements
cur = conn.cursor()

# Execute statement
print("Create database " + database_name)
cur.execute(sql_stmt)

We willen in de database ruimtelijke data op kunnen slaan en ruimtelijke analyses kunnen uitvoeren. Hiervoor moeten we de database uitbreiding [PostGIS](https://postgis.net/) installeren. PostGIS biedt het volgende:

* Geometry datatype voor de opslag van punten, lijnen en vlakken
* Ruimtelijke index om snel te kunnen zoeken op basis van locatie
* Functies om ruimtelijke analyses uit te voeren (bijv. zoeken binnen een gebied of een buffer om een punt berekenen)

De PostGIS uitbreiding is als volgt te installeren:


In [ ]:
# Build sql statement to install PostGIS extension
sql_stmt = 'CREATE EXTENSION POSTGIS'

# Get a cursor to execute SQL statements
cur = conn.cursor()

# Execute statement
print("Install PostGIS extension")
cur.execute(sql_stmt)

De volgende stap is het creëren van de tabellen. Uit het datamodel is af te leiden dat er een foreign key relatie bestaat tussen de tabellen _meteostation_ en _waarneming_. Elke waarnemening is gedaan op een meteostation. Daarom moeten we eerst de tabel _meteostation_ maken en daarna de tabel _waarneming_.

De tabel _meteostation_ heeft de volgende kolommen. Per kolom is het datatype aangegeven:

* id integer 
* hoogte real 
* naam character varying 
* eigenaar character varying 
* geom geometry(Geometry, 4326)

De kolom _id_ is de primary key. Alle kolommen behalve _eigenaar_ zijn verplicht (_not null_). Bij de _geometry_ kolom is aangegeven dat alle type geometrieën opgeslagen kunnen worden (punten, lijnen en vlakken) en dat het coördinaatsysteem EPSG:4326 (WGS84) moet zijn. Op de _geom_ kolom maken we een ruimtelijke index aan om snel te kunnen zoeken.

**Let op**: Gebruik geen hoofdletters of spaties in de tabel- en kolomnamen. Dit kan voor problemen zorgen.

De tabel is met het volgende script te creëren:

In [ ]:
# Get a cursor to execute SQL statements
cur = conn.cursor()

# Build SQL statement to create table
sql_stmt = 'CREATE TABLE public.meteostation'
sql_stmt = sql_stmt + '( id integer NOT NULL'
sql_stmt = sql_stmt + ', hoogte real NOT NULL' 
sql_stmt = sql_stmt + ', naam character varying NOT NULL'
sql_stmt = sql_stmt + ', eigenaar character varying' 
sql_stmt = sql_stmt + ', geom geometry(Geometry,4326) NOT NULL'
sql_stmt = sql_stmt + ', CONSTRAINT meteostation_pkey PRIMARY KEY (id)'
sql_stmt = sql_stmt + ')'
sql_stmt = sql_stmt + ' WITH (OIDS = FALSE) TABLESPACE pg_default'

# Execute statement
cur.execute(sql_stmt)

# Build SQL statement to create spatial index
sql_stmt = 'CREATE INDEX meteostation_geom_ix'
sql_stmt = sql_stmt + ' ON public.meteostation USING gist(geom)'
sql_stmt = sql_stmt + ' TABLESPACE pg_default'

# Execute statement
cur.execute(sql_stmt)

De tabel _waarneming_ heeft de volgende kolommen. Per kolom is het datatype aangegeven:

* id integer 
* datum date
* gemiddelde_temperatuur real
* meteostation_id integer

De kolom _id_ is de primary key. Alle kolommen zijn verplicht (_not null_). De kolom _meteostation\_id_ is de foreign key naar de tabel _meteostation_. 

De tabel is met het volgende script te creëren:

In [ ]:
# Get a cursor to execute SQL statements
cur = conn.cursor()

# Build SQL statement to create table
sql_stmt = 'CREATE TABLE public.waarneming'
sql_stmt = sql_stmt + '( id integer NOT NULL'
sql_stmt = sql_stmt + ', datum date NOT NULL' 
sql_stmt = sql_stmt + ', gemiddelde_temperatuur real NOT NULL'
sql_stmt = sql_stmt + ', meteostation_id integer NOT NULL' 
sql_stmt = sql_stmt + ', CONSTRAINT waarneming_pkey PRIMARY KEY (datum, meteostation_id),'
sql_stmt = sql_stmt + ', CONSTRAINT meteostation_id_fkey FOREIGN KEY (meteostation_id)'
sql_stmt = sql_stmt + '  REFERENCES public.meteostation (id) MATCH SIMPLE'
sql_stmt = sql_stmt + '  ON UPDATE NO ACTION'
sql_stmt = sql_stmt + '  ON DELETE NO ACTION'
sql_stmt = sql_stmt + ')'
sql_stmt = sql_stmt + ' WITH (OIDS = FALSE) TABLESPACE pg_default'

# Execute statement
cur.execute(sql_stmt)

# Build SQL statement to create FK index
sql_stmt = 'CREATE INDEX fki_meteostation_id_fkey'
sql_stmt = sql_stmt + ' ON public.waarneming USING btree(meteostation_id)'
sql_stmt = sql_stmt + ' TABLESPACE pg_default'

# Execute statement
cur.execute(sql_stmt)

De tabel _provincie_ heeft de volgende kolommen. Per kolom is het datatype aangegeven:

* id integer 
* naam character varying 
* geom geometry(Geometry, 4326)

De kolom _id_ is de primary key. Alle kolommen zijn verplicht (_not null_). Bij de _geometry_ kolom is aangegeven dat alle type geometrieën opgeslagen kunnen worden (punten, lijnen en vlakken) en dat het coördinaatsysteem [EPSG:4326](https://epsg.io/4326) (WGS84) moet zijn. Op de _geom_ kolom maken we een ruimtelijke index aan om snel te kunnen zoeken.

De tabel is met het volgende script te creëren:

In [ ]:
# Get a cursor to execute SQL statements
cur = conn.cursor()

# Build SQL statement to create table
sql_stmt = 'CREATE TABLE public.provincie'
sql_stmt = sql_stmt + '( id integer NOT NULL'
sql_stmt = sql_stmt + ', naam character varying NOT NULL'
sql_stmt = sql_stmt + ', geom geometry(Geometry,4326) NOT NULL'
sql_stmt = sql_stmt + ', CONSTRAINT provincie_pkey PRIMARY KEY (id)'
sql_stmt = sql_stmt + ')'
sql_stmt = sql_stmt + ' WITH (OIDS = FALSE) TABLESPACE pg_default'

# Execute statement
cur.execute(sql_stmt)

# Build SQL statement to create spatial index
sql_stmt = 'CREATE INDEX provincie_geom_ix'
sql_stmt = sql_stmt + ' ON public.provincie USING gist(geom)'
sql_stmt = sql_stmt + ' TABLESPACE pg_default'

# Execute statement
cur.execute(sql_stmt)

**Let op**: Voor het maken van de database is _autocommit_ aangezet. Dit betekent dat elk commando direct op de database uitgevoerd wordt en niet expliciet hoeft te worden gecommit. Een nieuwe database is direct voor alle gebruikers zichtbaar. Als je DML statements gaat uitvoeren is het gebruikelijk om de autocommit uit te zetten. Dit betekent dat je alle DML statements (insert, update of delete) expliciet moet committen voordat de wijzigen in de database doorgevoerd worden en voor andere gebruikers zichtbaar zijn. Deze manier van werken maakt het mogelijk om met transacties (meerdere SQL statements achter elkaar) te werken die in zijn geheel moeten worden doorgevoerd of moeten worden teruggedraaid. 

Je zet de autocommit af met het volgende statement:

In [ ]:
# Set autocommit off
conn.set_isolation_level(ISOLATION_LEVEL_DEFAULT)